In [11]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [12]:
frame = None
roiPts = []
inputMode = False

In [35]:
def select_ROI(event, x, y, flags, param):
    global frame, roiPts, inputMode
    print('select')
    if inputMode and event == cv2.EVENT_LBUTTONDBLCLK and len(roiPts)<4:
        roiPts.append((x,y))
        cv2.circle(frame,(x,y),4,(0,255,0),2)
        cv2.imshow('frame',frame)
        

In [36]:
def main():
    global frame, roiPts, inputMode
    
    cam = cv2.VideoCapture(0)
    
    cv2.namedWindow('frame')
    cv2.setMouseCallback('frame', select_ROI)
    
    termination = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,1)
    roibox = None
    
    while True:
        (ret, frame) = cam.read()
        if ret == False:
            break
        
        if roibox is not None:
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            backproj = cv2.calcBackProject([hsv],[0],roiHist,[0,180],1)
            print('roibox')
            (r,roibox) = cv2.CamShift(backproj, roibox, termination)
            pts = np.int0(cv2.boxPoints(r))
            cv2.polylines(frame,[pts],True,(0,255,0),2)
            
        cv2.imshow('frame', frame)
        key = cv2.waitKey(1) & 0xFF 
        if key == ord('a') & len(roiPts)<4:
            inputMode = True
            orig = frame.copy()
            
            while len(roiPts)<4:
                cv2.imshow('frame', frame)
                cv2.waitKey(0)
                
            roiPts = np.array(roiPts)
            s = roiPts.sum(axis=1)
            tl = roiPts[np.argmin(s)]
            br = roiPts[np.argmax(s)]
            
            
            roi = orig[tl[1]:br[1],tl[0]:br[0]]
            roi = cv2.cvtColor(orig, cv2.COLOR_BGR2HSV)
            
            
            roiHist = cv2.calcHist([roi],[0],None,[16],[0,180])
            roiHist = cv2.normalize(roiHist,roiHist,0,255, cv2.NORM_MINMAX)
            roibox = (tl[0],tl[1],br[0],br[1])
            pause(4000)
        elif key== ord('q'):
            break
            
    cam.release()
    cv2.destroyAllWindows()

In [37]:
if __name__ == "__main__":
    main()

select
select
select
select
select
select
select
select
select
select
select
select
select
select
select
select
select
select


In [24]:
def draw_circle(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        cv2.circle(img,(x,y),100,(255,0,0),-1)

# Create a black image, a window and bind the function to window
img = np.zeros((512,512,3), np.uint8)
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    if cv2.waitKey(20) & 0xFF == 27:
        break
cv2.destroyAllWindows()